# MNIST Embeddings

## Imports and Hyper-parameters 

In [1]:
import torch 
import os 
import numpy as np 
from torchvision import transforms 
from torchvision.datasets import MNIST 
from torchvision import models 

print(os.getcwd())
root = os.getcwd()

from DeepFeatures import DeepFeatures

/home/praveens/Desktop/synthetic_biometrics/visualize_embeddings


In [2]:
BATCH_SIZE = 64
DATA_FOLDER = root + '/MNIST'
IMAGES_FOLDER = root + '/Outputs/MNIST/Images'
EMBEDS_FOLDER = root + '/Outputs/MNIST/Embeds'
TENSORBOARD_FOLDER = root + '/Outputs/Tensorboard'
EXPERIMENT_NAME = 'mnist_embeds_vgg16'
device = 'cpu'

## Create Dataloader

In [3]:
def stack(tensor, times=3):
    return (torch.cat([tensor]*times, dim=0))

transformations = transforms.Compose([transforms.Resize((221, 221)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485], std=[0.229]),
                                      stack
                                    ])

mnist_data = MNIST(root=r'./MNIST',
                   download=False,
                   transform=transformations)

data_loader = torch.utils.data.DataLoader(mnist_data,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

In [4]:
vgg16 = models.vgg16(pretrained=True).to(device)

# class Identity(torch.nn.Module):
#     def __init__(self):
#         super(Identity, self).__init__()
    
#     def forward(self, x):
#         return x

vgg16.classifier = vgg16.classifier[0:4]
vgg16.eval()

/home/praveens/miniconda3/envs/viz_emb/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/praveens/miniconda3/envs/viz_emb/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Initialize Tensorboard Logging Class 

In [5]:
deep_features = DeepFeatures(model=vgg16, 
                             images_folder=IMAGES_FOLDER, 
                             embeds_folder=EMBEDS_FOLDER, 
                             tensorboard_folder=TENSORBOARD_FOLDER, 
                             experiment_name=EXPERIMENT_NAME)

In [6]:
images, labels = next(iter(data_loader))

In [7]:
e = deep_features.generate_embeddings(images)

In [8]:
e.shape

torch.Size([64, 4096])

In [9]:

x1 = torch.zeros(4096)
class1 = torch.tensor(6)
class1_count = 0

x2 = torch.zeros(4096)
class2 = torch.tensor(1)
class2_count = 0

for index, (image, label) in enumerate(zip(images, labels)):
    if label == class1:
        x1 += e[index]
        class1_count += 1
    if label == class2:
        x2 += e[index]
        class2_count += 1

In [10]:
x1 = x1/class1_count
x2 = x2/class2_count

In [11]:
d = (x1/torch.norm(x1)) - (x2/torch.norm(x2))

In [12]:
d = d.reshape(-1, 1)

In [13]:
d.shape

torch.Size([4096, 1])

In [14]:
Q, R = torch.linalg.qr(d)

In [15]:
Q.shape

torch.Size([4096, 1])

In [16]:
Q_inv = Q.T

In [17]:
Q_inv.shape

torch.Size([1, 4096])

In [18]:
S = torch.eye(4096)

In [19]:
S[0] = 0
S

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])

In [20]:
vgg16.classifier[3]

Linear(in_features=4096, out_features=4096, bias=True)

In [21]:
for name, param in vgg16.named_parameters():
    print(name)

features.0.weight
features.0.bias
features.2.weight
features.2.bias
features.5.weight
features.5.bias
features.7.weight
features.7.bias
features.10.weight
features.10.bias
features.12.weight
features.12.bias
features.14.weight
features.14.bias
features.17.weight
features.17.bias
features.19.weight
features.19.bias
features.21.weight
features.21.bias
features.24.weight
features.24.bias
features.26.weight
features.26.bias
features.28.weight
features.28.bias
classifier.0.weight
classifier.0.bias
classifier.3.weight
classifier.3.bias


In [22]:
last_layer = list(vgg16.parameters())[-2]
last_bias = list(vgg16.parameters())[-1]

In [23]:
last_bias.shape

torch.Size([4096])

In [24]:
last_layer = last_layer*Q*S*Q_inv

## Write Embeddings to Tensorboard

In [25]:
batch_images, batch_labels = next(iter(data_loader))
deep_features.write_embeddings(x=batch_images.to(device))

True

In [26]:
deep_features.create_tensorboard_log()

/home/praveens/Desktop/synthetic_biometrics/visualize_embeddings/DeepFeatures.py:94: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /home/builder/mc3/envs/pytorch-build/envs/pytorch-build/conda-bld/pytorch_1673601922403/work/torch/csrc/utils/tensor_new.cpp:201.)
  all_embeds = torch.Tensor(all_embeds)


torch.Size([64, 4096])
torch.Size([64, 3, 28, 28])
